In [1]:
import time
import pandas as pd
import numpy as np
import lightgbm as lgb
import config
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
###组合信息
timeFeatList = [
    ['user_id','day'],
    ['user_id','day','hour'],
    ['item_id','day'],
    ['item_id','day','hour'],
    ['shop_id','day'],
    ['shop_id','day','hour'],
    ['item_brand_id','day'],
    ['item_brand_id','day','hour'],
    ['item_city_id','day'],
    ['item_city_id','day','hour'],
]

In [3]:
def timestamp_datetime(value):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value))

def time_feat(df,featList,featName):
    '''scaler = StandardScaler()
    tmp = df.groupby(featList).size().reset_index().rename(columns={0:featName})
    tmp[featName] = scaler.fit_transform(tmp[featName].values.reshape(-1,1))
    df = df.merge(tmp,'left',on=featList)'''
    df[featName] = df.groupby(featList)['context_timestamp'].rank(method='first')   
    return df

def process(df):
    df['time'] = df.context_timestamp.apply(timestamp_datetime)
    df['day'] = df.time.apply(lambda x: int(x[8:10]))
    df['hour'] = df.time.apply(lambda x: int(x[11:13]))
    for lst in timeFeatList:
        df = time_feat(df,lst,'_'.join(lst))
    df['item_property_list_clean'] = df['item_property_list'].apply(lambda x:';'.join(sorted(set(str(x).split(';')))))
    for i in range(3):
        df['item_category_list_bin%d'%i] = df['item_category_list'].apply(lambda x: x.split(';')[i] if len(x.split(';'))>i else -1)
    df["missing_feat"] = np.sum((df == -1).values, axis=1)
    return df
def get_onehot(df,field):
    one = OneHotEncoder()
    lb = LabelEncoder()
    tmp = lb.fit_transform((list(df[field])))
    one.fit(tmp.reshape(-1,1))
    oneOut = one.transform(lb.transform(df[field]).reshape(-1,1))
    return oneOut

In [4]:
dfTrain = pd.read_table(config.TRAIN_FILE,sep=' ')
dfTrain.drop_duplicates(inplace=True)
dfTest = pd.read_table(config.TEST_FILE,sep=' ')

dfTrain = process(dfTrain)
dfTest = process(dfTest)

dfAll = pd.concat([dfTrain,dfTest],axis=0)
trainNum = dfTrain.shape[0]
dfAll.shape

(496482, 45)

<font color=#0099ff size=5 face="黑体">OneHot稀疏矩阵</font>

In [5]:
cutoff = 5
count = CountVectorizer(ngram_range=(1,1))
sparse_merge = count.fit_transform(dfAll['item_property_list_clean'])
for field in config.CATEGORICAL_COLS:
    tmp = get_onehot(dfAll,field)
    try:
        sparse_merge = hstack((sparse_merge,tmp)).tocsr()
    except:
        sparse_merge = tmp
           
sparse_merge = sparse_merge[:, np.array(np.clip(sparse_merge[:dfTrain.shape[0],:].getnnz(axis=0) -cutoff, 0, 1), dtype=bool)]

TypeError: '>' not supported between instances of 'int' and 'str'

In [6]:
sparse_merge = sparse_merge[:, np.array(np.clip(sparse_merge[:dfTrain.shape[0],:].getnnz(axis=0) -cutoff, 0, 1), dtype=bool)]

In [7]:
denseFeatList = config.NUMERIC_COLS
#+['_'.join(lst) for lst in timeFeatList]
denseFeat = dfAll[denseFeatList].values
sparse_merge = hstack((sparse_merge,denseFeat)).tocsr()

In [8]:
train_idx = dfTrain.loc[dfTrain['day']<24].index
valid_idx = dfTrain.loc[dfTrain['day']==24].index
Xi_train_, y_train_ = sparse_merge[list(train_idx),:],dfTrain.loc[train_idx,'is_trade']
Xi_valid_, y_valid_ = sparse_merge[list(valid_idx),:],dfTrain.loc[valid_idx,'is_trade']
Xi_test_ = sparse_merge[trainNum:,:]

<font color=#0099ff size=5 face="黑体">WOE筛选变量</font>

In [ ]:
from woe import calc_nominal_woe
from itertools import chain, combinations
import pickle

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

def feat_select(df,featList,order=[1,3],cutoff=0.1):
    dfTrain = df.copy()
    subset = powerset(featList)
    selected = {}
    for sub in subset:
        sub=list(sub)
        if len(sub)<order[0] or len(sub)>order[1]:
            continue
        if len(sub)==1:
            tmp_woe = calc_nominal_woe(df,sub[0],'is_trade',bins=10,small=0.001)
        else:
            df['test'] = df[sub[0]]
            for i in range(1,len(sub)):
                df['test'] = df[['test',sub[i]]].apply(lambda x:str(x[0])+'_'+str(x[1]),axis=1)
            tmp_woe = calc_nominal_woe(df,'test','is_trade',bins=10,small=0.001)
        if tmp_woe[-1]>=cutoff:
            print(sub,tmp_woe[-1])
            selected['_'.join(sub)] = tmp_woe[0]
    return selected
            


In [ ]:
tt = feat_select(dfTrain,config.CATEGORICAL_COLS,[3,3])

In [ ]:
with open('../../Data/advertisment/Cache/WOE.pkl','rb') as f:
    woe_dict = f.load()
woe_dict=dict(woe_dict.items()+tt.items())


In [ ]:
with open('../../Data/advertisment/Cache/WOE.pkl','wb') as f:
    pickle.dump(woe_dict,f)

<font color=#0099ff size=5 face="黑体">根据比值生成特征</font>

In [ ]:
for var in dfTrain.columns:
    if var =='is_trade':
        continue
    else:
        dfAll = dfAll.merge(dfTrain.groupby(var)['is_trade'].mean().reset_index().rename(columns={'is_trade':var+'_rate'}),'left',left_on=var,right_on=var)
        dfAll[var+'_rate'].fillna(dfTrain['is_trade'].mean(),inplace=True)

In [ ]:
features = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
                'item_collected_level', 'item_pv_level', 'user_gender_id', 'user_occupation_id',
                'user_age_level', 'user_star_level', 'user_query_day', 'user_query_day_hour',
                'context_page_id', 'hour', 'shop_id', 'shop_review_num_level', 'shop_star_level',
                'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description','item_property_list_clean'
                ]+['_'.join(lst) for lst in timeFeatList]

features = [var+'_rate' for var in features]

In [ ]:
train_idx = dfTrain.loc[dfTrain['day']<24].index
valid_idx = dfTrain.loc[dfTrain['day']==24].index
Xi_train_, y_train_ = dfAll.loc[list(train_idx),features],dfTrain.loc[train_idx,'is_trade']
Xi_valid_, y_valid_ = dfAll.loc[list(valid_idx),features],dfTrain.loc[valid_idx,'is_trade']
Xi_test_ = dfAll.loc[trainNum:,features]

<font color=#0099ff size=5 face="黑体">模型</font>

In [13]:
clf = lgb.LGBMClassifier(num_leaves=200, max_depth=20, n_estimators=80, n_jobs=20)
clf.fit(Xi_train_, y_train_,
        categorical_feature=[])
y_score_ = clf.predict_proba(Xi_valid_,)[:, 1]
print(log_loss(y_valid_, y_score_))

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


0.0860146912341


In [ ]:
Xi_finnal_ ,y_finnal_ = np.vstack(Xi_train_,Xi_valid_),np.vstack(y_train_,y_valid_)
clf.fit(Xi_finnal_, y_finnal_,
        categorical_feature=[])

y_test_meta = np.zeros((dfTest.shape[0], 1), dtype=float)
y_test_meta[:,0] += clf.predict_proba(Xi_test_)[:,1]
submit = pd.DataFrame({'instance_id':dfTest['instance_id'],'predicted_score':y_test_meta[:,0]})
submit.to_csv('../../Submission/advertisement/gbm_sparse_0328.txt', sep=" ", index=False, line_terminator='\n')

In [9]:
y_test_meta = np.zeros((dfTest.shape[0], 1), dtype=float)

In [10]:
submit = pd.DataFrame({'instance_id':dfTest['instance_id'],'predicted_score':y_test_meta[:,0]})

In [12]:
submit['predicted_score'] = 1.0
submit.to_csv('../../Submission/advertisement/test_0330.txt', sep=" ", index=False, line_terminator='\n')